In [7]:
import pandas as pd
import openpyxl

# 処理対象のファイル名のリストを格納
file_list = ['2022_年間売上表.xlsx','2023_年間売上表.xlsx']

# 結合後のデータを格納するための空のデータフレームを作成
df = pd.DataFrame()

# ファイルを一つずつ処理する
for i,file_name in enumerate(file_list):
    # 最初のファイルのみヘッダーを読み込む、それ以外はヘッダーを無視
    if i == 0:
        temp_df = pd.read_excel(file_name)
    else:
        temp_df = pd.read_excel(file_name,header =None)
        temp_df.columns = ['売上年','商品','金額（千円）'] # 後続ファイルにヘッダーを再設定

    # データフレームを連結
    df = pd.concat([df,temp_df],ignore_index =True)

# データフレーム内の重複するヘッダーを削除
df = df[~df['売上年'].isin(['売上年'])]

# 「商品」「売上年」でグループ化し、金額を合計
grouped_df = df.groupby(['商品', '売上年']).agg({'金額（千円）':'sum'})

# マルチインデックスを解消し、通常のFataFrameに変換
result_df = grouped_df.reset_index()

# 新しいExcelファイルに書き出す
result_df.to_excel('売上表.xlsx', index=False)


# 書き出したExcelファイルを開いて編集
workbook = openpyxl.load_workbook('売上表.xlsx')

# ワークシート名を取得
worksheet_name = workbook.sheetnames[0]

# ワークシートを指定
worksheet = workbook[worksheet_name]

# 1行目のすべてのセルの背景色をグレーに設定
for cell in worksheet[1]:
    cell.fill = openpyxl.styles.PatternFill(start_color = "F2F2F2",end_color = "F2F2F2", fill_type="solid")

# 変更を保存
workbook.save('売上表.xlsx')
